In [1]:
import geopandas as gpd
from geopandas.tools import sjoin
import pandas as pd
import numpy as np
import pyproj    
import shapely
import shapely.ops as ops
from functools import partial

In [19]:
# ISL Inference on Labelled Chips
tp_predictions_geo = "/Users/purgatorid/Downloads/true_positive_ISL_labelled.geojson"
fp_predictions_geo = "/Users/purgatorid/Downloads/false_positive_ISL_labelled.geojson"

# ISL Inference on entire CB
# all_predictions_geo = "/Users/purgatorid/Downloads/CB_ISL_Full_Predict_05242021.geojson"
# all_training_geo = "/Users/purgatorid/Downloads/ISL_Training_Labels.geojson"
true_labels_from_inference_geo = "/Users/purgatorid/Documents/GitHub/Project Canopy/cb_feature_detection/analytics/true_ISL_labels_from_inference.geojson"
new_predictions_from_inference_geo = "/Users/purgatorid/Documents/GitHub/Project Canopy/cb_feature_detection/analytics/new_ISL_predictions_from_inference.geojson"

# Orphan Detection 

# orphan_predictions_geo = "/Users/purgatorid/Downloads/PA_all.geojson"
orphan_predictions_true_geo = "/Users/purgatorid/Downloads/pred_orphans.geojson"
orphan_predictions_false_geo = "/Users/purgatorid/Downloads/pred_non_orphans.geojson"

# Backup Files

# cb_boundary = "/Users/user/Downloads/Congo_Basin_Boundary_no_islands_v5.geojson"
osm_cb_clipped_geo = "/Users/purgatorid/Downloads/osm_relevant_clipped.geojson"

In [3]:
tp_predictions = gpd.read_file(tp_predictions_geo)
fp_predictions = gpd.read_file(fp_predictions_geo)

# all_predictions = gpd.read_file(all_predictions_geo)
# training_labels = gpd.read_file(all_training_geo)

true_orphan_predictions = gpd.read_file(orphan_predictions_true_geo)
false_orphan_predictions = gpd.read_file(orphan_predictions_false_geo)


In [20]:
true_labels_from_inference = gpd.read_file(true_labels_from_inference_geo)
new_predictions_from_inference = gpd.read_file(new_predictions_from_inference_geo)

In [4]:
osm_clipped = gpd.read_file(osm_cb_clipped_geo)

In [152]:
def convert_to_4326(gdf):

    gdf = gdf.set_crs(epsg=3857)
    gdf = gdf.to_crs(epsg=4326)
    return gdf

true_orphan_predictions = convert_to_4326(true_orphan_predictions)
false_orphan_predictions = convert_to_4326(false_orphan_predictions)


#### Clip to CB Boundary - Already Done

In [44]:
# osm_clipped = gpd.clip(osm_full, cb_boundary)

In [45]:
# osm_clipped.to_file("osm_relevant_clipped.geojson", driver="GeoJSON")

#### % of Training Chips Detected by Prediction

In [21]:
# training_lines_match = sjoin(training_labels, all_predictions, how="inner")["geometry"]

In [22]:
# training_lines_match.nunique() / training_labels.shape[0]

### Chips in Training Set - Already Exported - See root directory and code above

In [23]:
# intersecting_polygons = sjoin(training_labels,all_predictions, how="right")

In [24]:
# true_labels_from_inference = gpd.GeoDataFrame(intersecting_polygons[pd.notna(intersecting_polygons["index_left"])]["geometry"])

In [25]:
# new_predictions_from_inference = gpd.GeoDataFrame(intersecting_polygons[pd.isnull(intersecting_polygons["index_left"])]["geometry"])

In [26]:
# true_labels_from_inference.to_file("true_ISL_labels_from_inference.geojson", driver="GeoJSON", index=True)

In [27]:
# new_predictions_from_inference.to_file("new_ISL_predictions_from_inference.geojson", driver="GeoJSON", index=True)

#### True Positive Analysis - See Function Below

In [28]:
# tp_osm = sjoin(true_positive_predictions,osm_clipped, how="right")

In [29]:
# tp_osm = tp_osm[pd.notna(tp_osm["index_left"])]

In [30]:
# tp_osm["tags"].reset_index().loc[0,"tags"]

In [31]:
# kv_tags = []
# for tag in tp_osm["tags"]:
#     kv_tag_list = []
#     for k in tag:
#         kv_tag = k + "_" + tag[k]
#         kv_tag_list.append(kv_tag)
#     kv_tags.append(kv_tag_list)

In [32]:
# tp_osm["tag_list"] = kv_tags

In [33]:
# tag_list_count = {}

# for tag_list in tp_osm["tag_list"]:
#     for tag in tag_list:
#         if tag not in tag_list_count.keys():
#             tag_list_count[tag] = 1 
#         else:
#             tag_list_count[tag] += 1

In [34]:
# tp_osm.shape

In [35]:
# tag_list_count

# {k: v for k, v in sorted(tag_list_count.items(), key=lambda item: item[1],reverse=True)}

# Function for Tag Count Intersect between OSM and Prediction Set

In [127]:
df1 = pd.DataFrame({"dataset":[]})

In [128]:
df1.append(pd.DataFrame({"dataset":[1,2]}))

,dataset
0,1.0
1,2.0


In [173]:
def gdf_osm_tag_count(gdf_dict,osm_gdf,filter_method="top_percent",filter_value=.8):
    
#     full_tag_percent_dict = {"dataset":[],"split":[],"tag":[],"count":[],"percent_containing":[]}
    
    full_df = pd.DataFrame({"dataset":[],"split":[],"tag":[],"count":[],"percent_containing":[]})
    
    for dataset in gdf_dict:
        
        for split in gdf_dict[dataset]:
            
            print(f'processing dataset {dataset} for split {split}')
            
            gdf_file = gdf_dict[dataset][split]
    
            tp_osm = sjoin(gdf_file,osm_gdf, how="right")

            tp_osm = tp_osm[pd.notna(tp_osm["index_left"])]

            kv_tags = []
            for tag in tp_osm["tags"]:
                kv_tag_list = []
                for k in tag:
                    kv_tag = k + "_" + tag[k]
                    kv_tag_list.append(kv_tag)
                kv_tags.append(kv_tag_list)

            tp_osm["tag_list"] = kv_tags

            tag_list_count = {}

            for tag_list in tp_osm["tag_list"]:
                for tag in tag_list:
                    if tag not in tag_list_count.keys():
                        tag_list_count[tag] = 1 
                    else:
                        tag_list_count[tag] += 1

            counts_sorted = {k: v for k, v in sorted(tag_list_count.items(), key=lambda item: item[1],reverse=True)}

            full_count = sum(counts_sorted.values())
            full_percent = [i/ full_count for i in counts_sorted.values()]
            
            if filter_method == "top_percent":
                
                sum_percent = 0 
                
                for i,percent in enumerate(full_percent):

                    sum_percent += percent
                    
                    
                    if sum_percent >= filter_value:
                        
                        dataset_list = [dataset] * len(full_percent)
                        split_list = [split] * len(full_percent)
                        
                        counts_sorted_df = pd.DataFrame({"dataset":dataset_list,"split":split_list,
                                                         "tag":counts_sorted.keys(),"count":counts_sorted.values(),
                                                         "percent_containing":full_percent}).head(i)
                
                        
                        full_df = full_df.append(counts_sorted_df)
                        
                        break
    
                
                
                

#             if filter_method == "filter_out_percent":
#                  counts_sorted_df_filtered = counts_sorted_df[counts_sorted_df["percent_containing"] < filter_out]

    return full_df

In [176]:
geo_files = {"full_cb_inference":{"true":true_labels_from_inference,"false":new_predictions_from_inference},
             "training_inference":{"true":tp_predictions,"false":fp_predictions},
             "orphan_sets":{"true":true_orphan_predictions,"false":false_orphan_predictions}}

df_full_tags = gdf_osm_tag_count(geo_files,osm_clipped)

processing dataset full_cb_inference for split true
full_cb_inference
processing dataset full_cb_inference for split false
full_cb_inference
processing dataset training_inference for split true
training_inference
processing dataset training_inference for split false
training_inference
processing dataset orphan_sets for split true
orphan_sets
processing dataset orphan_sets for split false
orphan_sets


In [203]:
df_full_tags["dataset"].value_counts()

orphan_sets           160
full_cb_inference      89
training_inference     70
Name: dataset, dtype: int64

In [202]:
df_false["dataset"].value_counts()

full_cb_inference    40
Name: dataset, dtype: int64

In [215]:
df_full_tags[df_full_tags["dataset"] != "orphan_sets"]

,dataset,split,tag,count,percent_containing
0,full_cb_inference,true,highway_track,4063.0,0.246168
1,full_cb_inference,true,access_forestry,3530.0,0.213875
2,full_cb_inference,true,source_CIRAD;Landsat,812.0,0.049197
3,full_cb_inference,true,source_Landsat;WRI,552.0,0.033444
4,full_cb_inference,true,start_date_2014,498.0,0.030173
...,...,...,...,...,...
47,training_inference,false,operator_Commune,8.0,0.003353
48,training_inference,false,ref_N 1,8.0,0.003353
49,training_inference,false,width_8,8.0,0.003353
50,training_inference,false,highway_trunk,8.0,0.003353


In [223]:
df_false = df_full_tags[(df_full_tags["split"] == "false") & (df_full_tags["dataset"] != "orphan_sets")].groupby('dataset').head(100)
df_true = df_full_tags[(df_full_tags["split"] == "true") & (df_full_tags["dataset"] != "orphan_sets")].groupby('dataset').head(100)

In [225]:
df_false[~df_false["tag"].isin(df_true["tag"])]["tag"].value_counts()

waterway_stream                                                                                                                2
source_WRI;DIAF;Africover                                                                                                      2
start_date_2000                                                                                                                2
boat_no                                                                                                                        2
waterway_river                                                                                                                 2
                                                                                                                              ..
source_ref_www.rgc.cd                                                                                                          1
note_Les limites de cette zone plantée seront révisées en utilisant une collection d'imagerie d'i

In [187]:
df_false

,dataset,split,tag,count,percent_containing
0,full_cb_inference,false,highway_track,28562.0,0.097311
1,full_cb_inference,false,access_forestry,25293.0,0.086173
2,full_cb_inference,false,waterway_river,17505.0,0.059640
3,full_cb_inference,false,source_Landsat;WRI,14837.0,0.050550
4,full_cb_inference,false,landuse_residential,14490.0,0.049367
...,...,...,...,...,...
60,orphan_sets,false,start_date_2009,521.0,0.002068
61,orphan_sets,false,tracktype_grade2,520.0,0.002064
62,orphan_sets,false,start_date_2013,505.0,0.002004
63,orphan_sets,false,source_bing,477.0,0.001893


In [ ]:
df1[~df1.index.isin(df2.index)]

df_full_tags[df]

In [159]:
tp_osm = sjoin(true_orphan_predictions,osm_clipped, how="right")

tp_osm = tp_osm[pd.notna(tp_osm["index_left"])]

In [161]:
true_orphan_predictions.head()

,is_orphan,geometry
0,True,"POLYGON ((25.28119 -3.40557, 25.28983 -3.40798..."
1,True,"POLYGON ((25.25050 -3.41571, 25.25913 -3.41812..."
2,True,"POLYGON ((25.25711 -3.15636, 25.26571 -3.15876..."
3,True,"POLYGON ((25.22033 -3.15541, 25.22893 -3.15781..."
4,True,"POLYGON ((25.26411 -2.99855, 25.27274 -3.00096..."


In [162]:
true_labels_from_inference

,index,geometry
0,6607,"POLYGON ((10.55094 1.27792, 10.55872 1.27346, ..."
1,6608,"POLYGON ((10.54651 1.27009, 10.55429 1.26564, ..."
2,6620,"POLYGON ((10.54316 1.28237, 10.55094 1.27792, ..."
3,6621,"POLYGON ((10.53874 1.27455, 10.54651 1.27009, ..."
4,6621,"POLYGON ((10.53874 1.27455, 10.54651 1.27009, ..."
...,...,...
9941,131242,"POLYGON ((17.11020 3.21104, 17.11845 3.20747, ..."
9942,131244,"POLYGON ((17.10195 3.21462, 17.11020 3.21104, ..."
9943,131245,"POLYGON ((17.09371 3.21820, 17.10195 3.21462, ..."
9944,131245,"POLYGON ((17.09371 3.21820, 17.10195 3.21462, ..."


In [131]:
new_df = gdf_osm_tag_count(new_predictions,osm_clipped,filter_out=1)

In [132]:
true_df = gdf_osm_tag_count(true_positive_predictions,osm_clipped,filter_out=1)

In [157]:
cross_df_outer = pd.merge(
    new_df,
    true_df,
    how="outer",
    on=None,
    left_on="tag",
    right_on="tag",
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
)

In [158]:
cross_df_outer.shape

(4108, 5)

#### Low Hanging Filter List - Represents of Occurring Tags

In [175]:
cross_df_outer["count_y"].isna().sum() / new_df.shape[0]

0.9690168333739937

In [173]:
new_df.shape[0]

4099

In [172]:
new_df.shape[0] - cross_df_outer["count_y"].isna().sum()

127

In [163]:
sum(filter_tags_misc["count_y"])

nan

#### Tags Appearing in Both Datasets

In [176]:
cross_df = cross_df_outer[cross_df_outer["count_y"].notna()]

In [177]:
cross_df

,tag,count_x,percent_containing_x,count_y,percent_containing_y
7,FIXME_check import,128.0,0.001009,3.0,0.000302
251,abandoned:date_before 2007,464.0,0.003657,15.0,0.001508
253,abandoned:date_before 2009,49.0,0.000386,25.0,0.002514
254,abandoned:date_before 2010,48.0,0.000378,3.0,0.000302
255,abandoned:date_before 2011,180.0,0.001419,6.0,0.000603
...,...,...,...,...,...
4056,way_secondary,812.0,0.006399,69.0,0.006937
4057,way_service,23.0,0.000181,15.0,0.001508
4058,way_tertiary,401.0,0.003160,20.0,0.002011
4059,way_track,1825.0,0.014382,336.0,0.033782


In [178]:
cross_df.sort_values(by="percent_containing_x",ascending=False)

,tag,count_x,percent_containing_x,count_y,percent_containing_y
620,highway_track,28562.0,0.225086,4063.0,0.408506
267,access_forestry,25293.0,0.199324,3530.0,0.354917
4049,waterway_river,17505.0,0.137950,34.0,0.003418
3793,source_Landsat;WRI,14837.0,0.116924,552.0,0.055500
1811,landuse_residential,14490.0,0.114190,19.0,0.001910
...,...,...,...,...,...
2984,name_Route Timberland,NaN,NaN,3.0,0.000302
3623,source:date_2007-05-10,NaN,NaN,8.0,0.000804
3779,source_Landast;JRC,NaN,NaN,6.0,0.000603
3949,start_date_2018,NaN,NaN,430.0,0.043233


* top 20 in each and see how much they show in the other
* top 80% occuring tags
* all tags above say 3% occurence 

#### New Chips Not in Training Set

In [4]:
intersecting_polygons = sjoin(predictions, training, how="left")

In [6]:
training

,width,color,note,geometry
0,1.0,"255,0,0,255",None,"LINESTRING (16.58639 2.19236, 16.58641 2.19236..."
1,1.0,"255,0,0,255",None,"LINESTRING (16.62630 2.19762, 16.62633 2.19762..."
2,1.0,"255,0,0,255",None,"LINESTRING (16.63438 2.18443, 16.63436 2.18443..."
3,1.0,"255,0,0,255",None,"LINESTRING (16.64634 2.19385, 16.64642 2.19385..."
4,1.0,"255,0,0,255",None,"LINESTRING (16.65006 2.19487, 16.65009 2.19487..."
...,...,...,...,...
4209,1.0,"255,0,0,255",None,"LINESTRING (17.35599 3.36944, 17.35602 3.36944..."
4210,1.0,"255,0,0,255",None,"LINESTRING (17.37302 3.36918, 17.37363 3.36918..."
4211,1.0,"255,0,0,255",None,"LINESTRING (17.38822 3.36912, 17.38880 3.36912..."
4212,1.0,"255,0,0,255",None,"LINESTRING (17.36942 3.36900, 17.36942 3.36897..."


In [8]:
values = {'index_right': "none"}
intersecting_polygons = intersecting_polygons.fillna(value=values)

In [9]:
intersecting_polygons = intersecting_polygons[intersecting_polygons["index_right"] == "none"]

#### 4% of Chips Comprised Training Chips  

In [10]:
intersecting_polygons.shape[0] / predictions.shape[0] 

0.9608159371237762

In [11]:
intersecting_polygons.shape

(126894, 5)

All Remaining Polygons Are Unique and not part of prediction set

In [15]:
intersecting_polygons["geometry"].nunique()

126894

In [13]:
predictions.shape[0] - intersecting_polygons.shape[0]

5175

In [14]:
predictions.shape[0]

132069

#### Area Ananlysis

In [ ]:
area_km_sq = 0

for item in intersecting_polygons.itertuples():
    geom = item[1]
    geom_area = ops.transform(
    partial(
        pyproj.transform,
        pyproj.Proj(init='EPSG:4326'),
        pyproj.Proj(
            proj='aea',
            lat_1=geom.bounds[1],
            lat_2=geom.bounds[3])),
    geom)

#     Print the area in m^2
#     print(geom)
#     print(geom_area.area / 10**6)
    area_km_sq += geom_area.area / 10**6

/Users/user/anaconda3/envs/conda-inference/lib/python3.9/site-packages/pyproj/crs/crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/Users/user/anaconda3/envs/conda-inference/lib/python3.9/site-packages/pyproj/crs/crs.py:306: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
/Users/user/anaconda3/envs/conda-inference/lib/python3.9/site-packages/pyproj/crs/crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<auth

# Sandbox

In [7]:
training

,width,color,note,geometry
0,1.0,"255,0,0,255",None,"LINESTRING (16.58639 2.19236, 16.58641 2.19236..."
1,1.0,"255,0,0,255",None,"LINESTRING (16.62630 2.19762, 16.62633 2.19762..."
2,1.0,"255,0,0,255",None,"LINESTRING (16.63438 2.18443, 16.63436 2.18443..."
3,1.0,"255,0,0,255",None,"LINESTRING (16.64634 2.19385, 16.64642 2.19385..."
4,1.0,"255,0,0,255",None,"LINESTRING (16.65006 2.19487, 16.65009 2.19487..."
...,...,...,...,...
4209,1.0,"255,0,0,255",None,"LINESTRING (17.35599 3.36944, 17.35602 3.36944..."
4210,1.0,"255,0,0,255",None,"LINESTRING (17.37302 3.36918, 17.37363 3.36918..."
4211,1.0,"255,0,0,255",None,"LINESTRING (17.38822 3.36912, 17.38880 3.36912..."
4212,1.0,"255,0,0,255",None,"LINESTRING (17.36942 3.36900, 17.36942 3.36897..."
